In [20]:
'''
本项目类似于翻译，encode，decode                  manPP,Swomen ==========>womenE

# S: Symbol that shows starting of decoding input
# E: Symbol that shows starting of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps

本项目中len(seq_data) = batch =6    n_class=v=26+3=29    max_len=n_step=5   h=128  
补齐max_len=5  

input 为 ['manPP', 'black','kingP', 'girlP', 'upPPP', 'highP']
input_batch= torch.Size([6, 5, 29]) = (b,max_len,v)

ouput 为  ['Swomen', 'Swhite', 'Squeen', 'SboyPP', 'SdownP', 'SlowPP']
output_batch= torch.Size([6, 6, 29]) = (b,max_len+1,v)

target 为  ['womenE', 'whiteE'', 'queenE'', 'boyPPE'', 'downPE'', 'lowPPE'']
target_batch =  torch.Size([6, 6]) = (b,max_len+1)



def forward(self, enc_input, enc_hidden, dec_input):         # enc_input: (m,b,v)    enc_hidden:(1,b,h),     dec_input: (m+1,b,v) 
        #（encode步骤）input和hidden运行RNN之后取其的最后attention：（1，b，h）
        _, enc_states = self.enc_cell(enc_input, enc_hidden)        #      self.enc_cell( (m,b,v) ,(1,b,h) )       # enc_states: = (1,b,h)
        
        #（decode步骤）将encode最后输出的attention和output   运行RNN之后取其输出outputs : (m+1,b,h)
        outputs, _ = self.dec_cell(dec_input, enc_states)          #  self.dec_cell( (m+1,b,v)  , (1,b,h))          # outputs : (m+1,b,h)

        然后(v,h) * (m+1,b,h) = (v,m+1,b) 
        (v,m+1,b)和target_batch运行softmax，迭代更新RNN的parameters
        
'''

"\n本项目类似于翻译，encode，decode                  manPP,Swomen ==========>womenE\n\n# S: Symbol that shows starting of decoding input\n# E: Symbol that shows starting of decoding output\n# P: Symbol that will fill in blank sequence if current batch data size is short than time steps\n\n本项目中len(seq_data) = batch =6    n_class=v=26+3=29    max_len=n_step=5   h=128  \n补齐max_len=5 \n\ninput 为 ['manPP', 'black','kingP', 'girlP', 'upPPP', 'highP']\ninput_batch= torch.Size([6, 5, 29]) = (b,max_len,v)\n\nouput 为  ['Swomen', 'Swhite', 'Squeen', 'SboyPP', 'SdownP', 'SlowPP']\noutput_batch= torch.Size([6, 6, 29]) = (b,max_len+1,v)\n\ntarget 为  ['womenE', 'whiteE'', 'queenE'', 'boyPPE'', 'downPE'', 'lowPPE'']\ntarget_batch =  torch.Size([6, 6]) = (b,max_len+1)\n\n\n\ndef forward(self, enc_input, enc_hidden, dec_input):         # enc_input: (m,b,v)    enc_hidden:(1,b,h),     dec_input: (m+1,b,v) \n        #（encode）input和hidden运行RNN之后区其的最后attention：（1，b，h）\n        _, enc_states = self.enc_cell(enc_input, 

In [2]:
# code by Tae Hwan Jung(Jeff Jung) @graykode
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

dtype = torch.FloatTensor


char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
num_dic = {n: i for i, n in enumerate(char_arr)}

seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

# Seq2Seq Parameter
n_step = 5
n_hidden = 128

n_class = len(num_dic)
batch_size = len(seq_data)

print(' char_arr=',char_arr)
print(' num_dic=',num_dic)
print(' n_class=',n_class)
print("batch_size = " ,batch_size)
print(' seq_data=',seq_data)


 char_arr= ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
 num_dic= {'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
 n_class= 29
batch_size =  6
 seq_data= [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]


In [16]:
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))
        print("0",seq[0],end="  ")
        input = [num_dic[n] for n in seq[0]]
        print("1",'S' + seq[1],end="  ")
        output = [num_dic[n] for n in ('S' + seq[1])]
        print("2",seq[1] +'E',end="            ")
        target = [num_dic[n] for n in (seq[1] +'E')]
    

        input_batch.append(np.eye(n_class)[input])      #input_batch= torch.Size([6, 5, 29])
        output_batch.append(np.eye(n_class)[output])  #output_batch= torch.Size([6, 6, 29])
        target_batch.append(target) # not one-hot        #target_batch =  torch.Size([6, 6])

    # make tensor
    return Variable(torch.Tensor(input_batch)), Variable(torch.Tensor(output_batch)), Variable(torch.LongTensor(target_batch))

print(' seq_data=',seq_data)
input_batch, output_batch, target_batch = make_batch(seq_data)
print(' seq_data=',seq_data)
print(' input_batch=',input_batch.size())
print(' output_batch=',output_batch.size())
print("target_batch = " ,target_batch.size())

 seq_data= [['manPP', 'women'], ['black', 'white'], ['kingP', 'queen'], ['girlP', 'boyPP'], ['upPPP', 'downP'], ['highP', 'lowPP']]
0 manPP  1 Swomen  2 womenE            0 black  1 Swhite  2 whiteE            0 kingP  1 Squeen  2 queenE            0 girlP  1 SboyPP  2 boyPPE            0 upPPP  1 SdownP  2 downPE            0 highP  1 SlowPP  2 lowPPE             seq_data= [['manPP', 'women'], ['black', 'white'], ['kingP', 'queen'], ['girlP', 'boyPP'], ['upPPP', 'downP'], ['highP', 'lowPP']]
 input_batch= torch.Size([6, 5, 29])
 output_batch= torch.Size([6, 6, 29])
target_batch =  torch.Size([6, 6])


In [9]:
# Model
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)   #(v,h)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.5)   #(v,h)
        self.fc = nn.Linear(n_hidden, n_class)  #(h,v) 

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = enc_input.transpose(0, 1) # enc_input: (m,b,v) 
        dec_input = dec_input.transpose(0, 1) # dec_input: (m+1,b,v) 
        
        # enc_states : [num_layers(=1) * num_directions(=1), b, h] = (1,b,h)
        _, enc_states = self.enc_cell(enc_input, enc_hidden)        #      self.enc_cell( (m,b,v) ,(1,b,h) )
        
        
        # outputs : [m+1(=6), b, num_directions(=1) * n_hidden(=128)] = (m+1,b,h)
        outputs, _ = self.dec_cell(dec_input, enc_states)          #  self.dec_cell( (m+1,b,v)  , (1,b,h))

        model = self.fc(outputs) #  (v,h) * (m+1,b,h) = (v,m+1,b) 
        return model




model = Seq2Seq()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    # make hidden shape [num_layers * num_directions, batch_size, n_hidden]
    hidden = Variable(torch.zeros(1, batch_size, n_hidden))       #(1,b,h)

    optimizer.zero_grad()
    # input_batch :  (b,m,v)
    # output_batch :  (b,m+1,v)
    # target_batch :   (b,m+1)
    output = model(input_batch, hidden, output_batch)    # model(  (b,m,v),    (1,b,h), (b,m+1,v)  )
    # output : (m+1,b,v) 
    output = output.transpose(0, 1) # [b, m+1, v]
    loss = 0        
    for i in range(0, len(target_batch)):     #len(target_batch) = b =6
        # output[i] : [max_len+1, n_class, target_batch[i] : max_len+1]
        loss += criterion(output[i], target_batch[i])
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.003676


In [19]:
# Test
def translate(word):
    print([[word, 'P' * len(word)]],end="       ")
    input_batch, output_batch, _ = make_batch([[word, 'P' * len(word)]])

    # make hidden shape [num_layers * num_directions, batch_size, n_hidden]
    hidden = Variable(torch.zeros(1, 1, n_hidden))
    output = model(input_batch, hidden, output_batch)
    # output : [m+1(=6), b(=1), v]

    predict = output.data.max(2, keepdim=True)[1] # select n_class dimension
    
    decoded = [char_arr[i] for i in predict]
    print(decoded)
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated.replace('P', '')

print('test')
print('man ->', translate('man'))
print('mans ->', translate('mans'))
print('king ->', translate('king'))
print('black ->', translate('black'))
print('upp ->', translate('upp'))

test
[['man', 'PPP']]       0 manPP  1 SPPPPP  2 PPPPPE            ['w', 'o', 'm', 'e', 'n', 'E']
man -> women
[['mans', 'PPPP']]       0 mansP  1 SPPPPP  2 PPPPPE            ['w', 'o', 'm', 'e', 'n', 'E']
mans -> women
[['king', 'PPPP']]       0 kingP  1 SPPPPP  2 PPPPPE            ['q', 'u', 'e', 'e', 'n', 'E']
king -> queen
[['black', 'PPPPP']]       0 black  1 SPPPPP  2 PPPPPE            ['w', 'h', 'i', 't', 'e', 'E']
black -> white
[['upp', 'PPP']]       0 uppPP  1 SPPPPP  2 PPPPPE            ['d', 'o', 'w', 'n', 'P', 'E']
upp -> down
